In [3]:
import pandas as pd
import numpy as np
import csv
# import xgboost as xgb
from tqdm import tqdm
from sklearn.svm import SVC
from sklearn import preprocessing#, decomposition, model_selection, metrics, pipeline
from nltk import word_tokenize
from sklearn.model_selection import cross_val_score
from nltk.corpus import stopwords
stop_words = stopwords.words('english')

In [2]:
train = pd.read_csv('./train.csv')
test = pd.read_csv('./test.csv')
# sample = pd.read_csv('../input/sample_submission.csv')
print(train.shape)
test.shape

(106445, 3)


(26610, 2)

In [16]:
# delete document with empty content
rm_idx = []
for i in range(len(test)):
    if len(test.Text[i]) <= 20:
        print(test.Text[i,])
        rm_idx.append(i)

print(rm_idx)

[4235, 4238, 4265, 8749, 8756, 8818, 18393, 18427, 19020, 25067, 27378, 27768, 31485, 31660, 31671, 31738, 40951, 41030, 41129, 41136, 41152, 41171, 41275, 41586, 42016, 45758, 45759, 45760, 45761, 50687, 51670, 51710, 52231, 52347, 52943, 66800, 70787, 78766, 92399, 92637, 94982, 97814, 98045, 101651, 101850, 101862, 102065, 102117, 102203]


In [4]:
# load the GloVe vectors in a dictionary:
embeddings_index = {}
f = open('./glove.42B.300d.txt', encoding="utf8")
for line in tqdm(f):
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

1917494it [02:21, 13571.78it/s]


Found 1917494 word vectors.


In [ ]:
import WordNet Lemmatizer from nltk 
from nltk.stem import WordNetLemmatizer 
wordnet_lemmatizer = WordNetLemmatizer() 
lines_with_lemmas=[] #stop words contain the set of stop words 
for line in lines: 
temp_line=[] 
for word in lines: 
    temp_line.append (wordnet_lemmatizer.lemmatize(word)) 
    string=’ ‘ 
    lines_with_lemmas.append(string.join(temp_line)) 
    lines=lines_with_lemmas

In [1]:
# this function creates a scaled vector for an entire document
def doc2vec(s):
    words = str(s).lower()
    words = word_tokenize(words) # tokenize the doc
    words = [w for w in words if not w in stop_words] # remove stop words
    words = [w for w in words if w.isalpha()] # remove numbers

    # create an array excluding stopwords and numbers
    M = []
    for w in words:
        try:
            M.append(embeddings_index[w])
        except:
            continue
    M = np.array(M)

    # for error document, convert into a 300d vector of 0
    v = M.sum(axis=0)
    if type(v) != np.ndarray:
        return np.zeros(300)
        
    return v / np.sqrt((v ** 2).sum())

In [5]:
xtrain = train.Text.values
ytrain = train.label
xtest = test.Text.values

# create sentence vectors using the above function for training and validation set
xtrain_glove = [sent2vec(x) for x in tqdm(xtrain)]
xtest_glove = [sent2vec(x) for x in tqdm(xtest)]

100%|███████████████████████████████████████████████████████████████████████████| 26610/26610 [00:50<00:00, 524.76it/s]


In [6]:
train_colname = []
test_colname = []
for i in range(300):
    name = "v" + str(i)
    train_colname.append(name)
    test_colname.append(name)
# print(train_colname)

In [7]:
train_output = pd.DataFrame.from_records(xtrain_glove)
test_output = pd.DataFrame.from_records(xtest_glove)

train_output.columns = train_colname
test_output.columns = test_colname

# train_output['id'] = train.ID
train_output['label'] = ytrain

# test_output['id'] = test.ID

In [11]:
# train_output = train_output[new_traincol]
# test_output = test_output[new_testcol]
print(train_output.shape)
print(test_output.shape)

(106445, 301)
(26610, 300)


In [9]:
train_output.to_csv('./train_glove_final.csv',index=False)
test_output.to_csv('./test_glove_final.csv',index=False)